Azure ML & Azure Databricks notebooks by Parashar Shah.

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

#Model Building

In [3]:
import os
import pprint
import numpy as np

from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [4]:
import azureml.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.10

In [5]:
# import the Workspace class and check the azureml SDK version
from azureml.core import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Found the config file in: /databricks/driver/aml_config/config.json
Workspace name: AMLSworkspace
Azure region: westeurope
Resource group: resgrpAMLS

In [6]:
#get the train and test datasets
train_data_path = "AdultCensusIncomeTrain"
test_data_path = "AdultCensusIncomeTest"

train = spark.read.parquet(train_data_path)
test = spark.read.parquet(test_data_path)

print("train: ({}, {})".format(train.count(), len(train.columns)))
print("test: ({}, {})".format(test.count(), len(test.columns)))

train.printSchema()

train: (25967, 15)
test: (6594, 15)
root
-- race: string (nullable = true)
-- capital_gain: integer (nullable = true)
-- workclass: string (nullable = true)
-- hours_per_week: integer (nullable = true)
-- native_country: string (nullable = true)
-- sex: string (nullable = true)
-- education: string (nullable = true)
-- age: integer (nullable = true)
-- education_num: integer (nullable = true)
-- marital_status: string (nullable = true)
-- capital_loss: integer (nullable = true)
-- occupation: string (nullable = true)
-- fnlwgt: integer (nullable = true)
-- relationship: string (nullable = true)
-- income: string (nullable = true)

#Define Model

In [8]:
label = "income"
dtypes = dict(train.dtypes)
dtypes.pop(label)

si_xvars = []
ohe_xvars = []
featureCols = []
for idx,key in enumerate(dtypes):
    if dtypes[key] == "string":
        featureCol = "-".join([key, "encoded"])
        featureCols.append(featureCol)
        
        tmpCol = "-".join([key, "tmp"])
        # string-index and one-hot encode the string column
        #https://spark.apache.org/docs/2.3.0/api/java/org/apache/spark/ml/feature/StringIndexer.html
        #handleInvalid: Param for how to handle invalid data (unseen labels or NULL values). 
        #Options are 'skip' (filter out rows with invalid data), 'error' (throw an error), 
        #or 'keep' (put invalid data in a special additional bucket, at index numLabels). Default: "error"
        si_xvars.append(StringIndexer(inputCol=key, outputCol=tmpCol, handleInvalid="skip"))
        ohe_xvars.append(OneHotEncoder(inputCol=tmpCol, outputCol=featureCol))
    else:
        featureCols.append(key)

# string-index the label column into a column named "label"
si_label = StringIndexer(inputCol=label, outputCol='label')

# assemble the encoded feature columns in to a column named "features"
assembler = VectorAssembler(inputCols=featureCols, outputCol="features")

In [9]:
from azureml.core.run import Run
from azureml.core.experiment import Experiment
import numpy as np
import os
import shutil

model_name = "AdultCensus_runHistory.mml"
model_dbfs = os.path.join("/dbfs", model_name)
run_history_name = 'spark-ml-notebook'

# start a training run by defining an experiment
myexperiment = Experiment(ws, "Ignite_AI_Talk")
root_run = myexperiment.start_logging()

# Regularization Rates - 
regs = [0.0001, 0.001, 0.01, 0.1]

# try a bunch of regularization rate in a Logistic Regression model
for reg in regs:
    print("Regularization rate: {}".format(reg))
    # create a bunch of child runs
    with root_run.child_run("reg-" + str(reg)) as run:
        # create a new Logistic Regression model.
        lr = LogisticRegression(regParam=reg)
        
        # put together the pipeline
        pipe = Pipeline(stages=[*si_xvars, *ohe_xvars, si_label, assembler, lr])

        # train the model
        model_p = pipe.fit(train)
        
        # make prediction
        pred = model_p.transform(test)
        
        # evaluate. note only 2 metrics are supported out of the box by Spark ML.
        bce = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction')
        au_roc = bce.setMetricName('areaUnderROC').evaluate(pred)
        au_prc = bce.setMetricName('areaUnderPR').evaluate(pred)

        print("Area under ROC: {}".format(au_roc))
        print("Area Under PR: {}".format(au_prc))
      
        # log reg, au_roc, au_prc and feature names in run history
        run.log("reg", reg)
        run.log("au_roc", au_roc)
        run.log("au_prc", au_prc)
        run.log_list("columns", train.columns)

        # save model
        model_p.write().overwrite().save(model_name)
        
        # upload the serialized model into run history record
        mdl, ext = model_name.split(".")
        model_zip = mdl + ".zip"
        shutil.make_archive(mdl, 'zip', model_dbfs)
        run.upload_file("outputs/" + model_name, model_zip)        
        #run.upload_file("outputs/" + model_name, path_or_stream = model_dbfs) #cannot deal with folders

        # now delete the serialized model from local folder since it is already uploaded to run history 
        shutil.rmtree(model_dbfs)
        os.remove(model_zip)
        
# Declare run completed
root_run.complete()
root_run_id = root_run.id
print ("run id:", root_run.id)

Regularization rate: 0.0001
Area under ROC: 0.9097083605819115
Area Under PR: 0.7632878176480774
Regularization rate: 0.001
Area under ROC: 0.9095016333650277
Area Under PR: 0.7621678874667115
Regularization rate: 0.01
Area under ROC: 0.9077819269927244
Area Under PR: 0.7533201198922834
Regularization rate: 0.1
Area under ROC: 0.8990925231800638
Area Under PR: 0.725793265379943
run id: ed9c1448-0f4e-46e0-826c-a26c385130e0

In [10]:
#Load all run metrics from run history into a dictionary object.
child_runs = {}

for r in root_run.get_children():
    child_runs[r.id] = r

In [11]:
metrics = root_run.get_metrics(recursive=True)
best_run_id = max(metrics, key = lambda k: metrics[k]['au_roc'])
best_run = child_runs[best_run_id]
print('Best run is:', best_run_id)
print('Metrics:', metrics[best_run_id]['au_roc'], metrics[best_run_id]['reg'])

Best run is: 6b80c8bb-2de3-470a-8a69-eecaad64cbb6
Metrics: 0.9097083605819115 0.0001

In [12]:
#Download the model from the best run to a local folder
best_model_file_name = "best_model.zip"
best_run.download_file(name = 'outputs/' + model_name, output_file_path = best_model_file_name)

#Model Evaluation

In [14]:
##unzip the model to dbfs (as load() seems to require that) and load it.
if os.path.isfile(model_dbfs) or os.path.isdir(model_dbfs):
    shutil.rmtree(model_dbfs)
shutil.unpack_archive(best_model_file_name, model_dbfs)

model_p_best = PipelineModel.load(model_name)

In [15]:
# make prediction
pred = model_p_best.transform(test)
output = pred[['hours_per_week','age','marital_status','income','prediction']]
display(output.limit(8))

hours_per_week,age,marital_status,income,prediction
10,60,Divorced,<=50K,0.0
25,23,Never-married,<=50K,0.0
40,27,Married-civ-spouse,<=50K,0.0
20,37,Divorced,<=50K,0.0
40,53,Divorced,<=50K,0.0
40,31,Married-civ-spouse,<=50K,0.0
30,26,Never-married,<=50K,0.0
40,36,Married-civ-spouse,<=50K,0.0


In [16]:
# evaluate. note only 2 metrics are supported out of the box by Spark ML.
bce = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction')
au_roc = bce.setMetricName('areaUnderROC').evaluate(pred)
au_prc = bce.setMetricName('areaUnderPR').evaluate(pred)

print("Area under ROC: {}".format(au_roc))
print("Area Under PR: {}".format(au_prc))

Area under ROC: 0.9097083605819115
Area Under PR: 0.7632878176480774

#Model Persistence

In [18]:
##NOTE: by default the model is saved to and loaded from /dbfs/ instead of cwd!
model_p_best.write().overwrite().save(model_name)
print("saved model to {}".format(model_dbfs))

saved model to /dbfs/AdultCensus_runHistory.mml

In [19]:
%sh

ls -la /dbfs/AdultCensus_runHistory.mml/*

/dbfs/AdultCensus_runHistory.mml/metadata:
total 0
drwxr-xr-x 1 root root 0 Jan 30 13:17 .
drwxr-xr-x 1 root root 0 Jan 30 13:17 ..
-rw-r--r-- 1 root root 736 Jan 30 13:17 part-00000
-rw-r--r-- 1 root root 0 Jan 30 13:17 _SUCCESS

/dbfs/AdultCensus_runHistory.mml/stages:
total 0
drwxr-xr-x 1 root root 0 Jan 30 13:17 .
drwxr-xr-x 1 root root 0 Jan 30 13:17 ..
drwxr-xr-x 1 root root 0 Jan 30 13:17 00_StringIndexer_e9eac5535e1f
drwxr-xr-x 1 root root 0 Jan 30 13:17 01_StringIndexer_8a8109bb0dcd
drwxr-xr-x 1 root root 0 Jan 30 13:17 02_StringIndexer_24367c6f8a41
drwxr-xr-x 1 root root 0 Jan 30 13:17 03_StringIndexer_61bb60473adb
drwxr-xr-x 1 root root 0 Jan 30 13:17 04_StringIndexer_6a560ce46480
drwxr-xr-x 1 root root 0 Jan 30 13:17 05_StringIndexer_0b3f84c6b4b6
drwxr-xr-x 1 root root 0 Jan 30 13:17 06_StringIndexer_fc8dbc613607
drwxr-xr-x 1 root root 0 Jan 30 13:17 07_StringIndexer_bb33dc169270
drwxr-xr-x 1 root root 0 Jan 30 13:17 08_OneHotEncoder_5d507dba507c
drwxr-xr-x 1 root root 0 Jan 30 13:17 09_OneHotEncoder_b522779dbd5d
drwxr-xr-x 1 root root 0 Jan 30 13:17 10_OneHotEncoder_55598cdb23de
drwxr-xr-x 1 root root 0 Jan 30 13:17 11_OneHotEncoder_fdf357861c67
drwxr-xr-x 1 root root 0 Jan 30 13:17 12_OneHotEncoder_2b5adc556ac5
drwxr-xr-x 1 root root 0 Jan 30 13:17 13_OneHotEncoder_3ed900233ad4
drwxr-xr-x 1 root root 0 Jan 30 13:17 14_OneHotEncoder_5c94756f39c7
drwxr-xr-x 1 root root 0 Jan 30 13:17 15_OneHotEncoder_536a0268d445
drwxr-xr-x 1 root root 0 Jan 30 13:17 16_StringIndexer_2d3028e65162
drwxr-xr-x 1 root root 0 Jan 30 13:18 17_VectorAssembler_55d435379165
drwxr-xr-x 1 root root 0 Jan 30 13:18 18_LogisticRegression_6dad8525703b

In [20]:
dbutils.notebook.exit("success")

success